# Attendance System

## 1. Training a New User

In [2]:
import cv2
import pickle
import numpy as np
import os
video=cv2.VideoCapture(0)
facedetect=cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

faces_data=[]

i=0

name=input("Enter Your Name: ")

while True:
    ret,frame=video.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray, 1.3 ,5)
    for (x,y,w,h) in faces:
        crop_img=frame[y:y+h, x:x+w, :]
        resized_img=cv2.resize(crop_img, (50,50))
        if len(faces_data)<=100 and i%10==0:
            faces_data.append(resized_img)
        i=i+1
        cv2.putText(frame, str(len(faces_data)), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (50,50,255), 1)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 1)
    cv2.imshow("Frame",frame)
    k=cv2.waitKey(1)
    if k==ord('q') or len(faces_data)==100:
        break
video.release()
cv2.destroyAllWindows()

faces_data=np.asarray(faces_data)
faces_data=faces_data.reshape(100, -1)


if 'names.pkl' not in os.listdir('data/'):
    names=[name]*100
    with open('data/names.pkl', 'wb') as f:
        pickle.dump(names, f)
else:
    with open('data/names.pkl', 'rb') as f:
        names=pickle.load(f)
    names=names+[name]*100
    with open('data/names.pkl', 'wb') as f:
        pickle.dump(names, f)

if 'faces_data.pkl' not in os.listdir('data/'):
    with open('data/faces_data.pkl', 'wb') as f:
        pickle.dump(faces_data, f)
else:
    with open('data/faces_data.pkl', 'rb') as f:
        faces=pickle.load(f)
    faces=np.append(faces, faces_data, axis=0)
    with open('data/faces_data.pkl', 'wb') as f:
        pickle.dump(faces, f)

Enter Your Name: Name


## 2. Facial Recognization Attendance System Update In Excel

In [10]:
import cv2
import pickle
import csv
from datetime import datetime, timedelta
from sklearn.neighbors import KNeighborsClassifier
from win32com.client import Dispatch
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
import csv
import os

def speak(str1):
    speak = Dispatch("SAPI.SpVoice")
    speak.Speak(str1)

def load_attendance_records(filepath):
    attendance_records = {}
    try:
        with open(filepath, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                user = row['NAME']
                last_time = datetime.strptime(row['TIME'], "%H:%M:%S")
                attendance_records[user] = last_time
    except FileNotFoundError:
        print("No existing attendance file found. Starting fresh.")
    except Exception as e:
        print(f"Error loading attendance records: {e}")
    return attendance_records

def append_attendance_record(filepath, name, time_str):
    file_exists = os.path.isfile(filepath)
    with open(filepath, "a", newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists:
            writer.writerow(['NAME', 'TIME'])  # Write the header if the file does not exist
        writer.writerow([name, time_str])

# Initialize camera and classifiers
video = cv2.VideoCapture(0)
facedetect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load data
with open('data/names.pkl', 'rb') as w:
    LABELS = pickle.load(w)
with open('data/faces_data.pkl', 'rb') as f:
    FACES = pickle.load(f)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(FACES, LABELS)

imgBackground = cv2.imread("background.png")

# Load existing attendance records
date = datetime.now().strftime("%d-%m-%Y")
attendance_file = f"Attendance/Attendance_{date}.csv"
attendance_records = load_attendance_records(attendance_file)

print("Loaded attendance records:", attendance_records)

while True:
    ret, frame = video.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facedetect.detectMultiScale(gray, 1.3, 5)
    
    current_time = datetime.now().time()
    for (x, y, w, h) in faces:
        crop_img = frame[y:y+h, x:x+w]
        resized_img = cv2.resize(crop_img, (50, 50)).flatten().reshape(1, -1)
        output = knn.predict(resized_img)
        label = str(output[0]) if output[0] in LABELS else "Unknown"
        should_record = True

        should_record = True  # Assume we should record the attendance

        if label in attendance_records:
            last_recorded_time = attendance_records[label].time()  # Extract time part
            current_datetime = datetime.combine(datetime.today(), current_time)
            last_recorded_datetime = datetime.combine(datetime.today(), last_recorded_time)
        
            time_diff_minutes = (current_datetime - last_recorded_datetime).total_seconds() / 60.0
        
            if time_diff_minutes < 10:
                should_record = False
        if should_record:
            print(f"Recording attendance for {label} at {current_time.strftime('%H:%M:%S')}")
            append_attendance_record(attendance_file, label, current_time.strftime("%H:%M:%S"))
            attendance_records[label] = current_time
            # Now, update the Google Sheet
            #update_google_sheet_from_csv(attendance_file)
        else:
            print(f"Skipping attendance for {label}, last recorded less than 10 minutes ago.")

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame", imgBackground)
    
    k = cv2.waitKey(1)
    if k == ord('o'):
        speak("Attendance Taken..")
    if k == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


Loaded attendance records: {'Name': datetime.datetime(1900, 1, 1, 15, 43, 39)}
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping attendance for Name, last recorded less than 10 minutes ago.
Skipping at

## 3. Loading CSV File

In [11]:
import pandas as pd
data = pd.read_csv('./Attendance/Attendance_25-04-2024.csv')
data.head()

,NAME,TIME
0,Name,15:43:39


## 4. Updating CSV File in MySQL Data Base

In [12]:
import pandas as pd
import mysql.connector as mysql
import warnings
warnings.filterwarnings("ignore")
from mysql.connector import Error

try:
    conn = mysql.connect(host='localhost', database='my_database', user='root', password='*************') # tips
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS data;') # tips_data
        print('Creating table....')
        # in the below line please pass the create table statement which you want #to create
        #tips_data(columnname1 datatype,columnname2 datatype .....)
        cursor.execute("CREATE TABLE data(name VARCHAR(255), time TIME)")
        print("Table is created....")
        #loop through the data frame
        for i,row in data.iterrows():
            #here %S means
            # tips.tips_data
            sql = "INSERT INTO my_database.data VALUES (%s,%s)" # how many columns are there that many %s
            cursor.execute(sql, tuple(row))
            #print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('my_database',)
Creating table....
Table is created....


In [13]:
data1=pd.read_sql("SELECT * FROM my_database.data",conn)
display(data1.head(50))

,name,time
0,Name,0 days 15:43:39


# To Connect to GDrive Code